In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
print(vectordb._collection.count())

In [ ]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

## RetrievalQA chain

In [2]:
from langchain.chains import RetrievalQA   


qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       retriver = vectordb.as_retriever())

result = qa_chain({"query": question})
result['result']

In [2]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

result = qa_chain({"query": "what are the topics in second lecture and if thre is any topic related to regression in the whole lecture and if yes what are they"})


## or we can use 

qa_chain = RetrievalQA.from_llm(
    llm,
    retriever=vectordb.as_retriever()
)
result = qa_chain({"query": "what are the topics in second lecture and if thre is any topic related to regression in the whole lecture and if yes what are they"})
await qa_chain.arun("summarize the third lecture")

## Prompt

In [1]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
qa_chain = RetrievalQA(llm,
                       retriever = vectordb.as_retriever(),
                       return_soruce_documents = True, # this easily make inspect the document, 
                       chain_type_kwargs = {"prompt":QA_CHAIN_PROMPT}
                       )


In [ ]:
question = "Is probability a class topic?"

result = qa_chain({"query": question})
result["result"]

## RetrievalQA chain types


- Map_reduce : 
    - In this technique, each of the individual documents passes into language model by itself to get answer, then those answers are composed into a final answer.
    - This involves many more call to language model, but iit has many advantages. It can opearte many individual documents. 
    - This technique is very slower and result is not always right,because its answering baseed each documents individually. Eg: if information or data spred across differnt documents, its doesnt have all in same context.        
- Refine     :
    - More accurate from above method, because its refining the output from model and going loop (if needed) until get good result
- Map_rerank :
    - other method
    

In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]